# XAI Neural Net Creation

## setup

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import os
import sys
currDir = os.path.dirname(os.path.realpath("__file__"))
rootDir = os.path.abspath(os.path.join(currDir, '..'))
sys.path.insert(1, rootDir)

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical

from src.data.make_dataset import make_dataset_1, make_dataset_2
from src.utils.file_utils import save_df, save_model

## generate artificial dataset

In [45]:
df1 = make_dataset_1()
df2 = make_dataset_2()
save_df(df1, "dataset1.csv")
save_df(df2, "dataset2.csv")

successfully generated dataset | num_rows: 2600000
successfully generated dataset | num_rows: 504000
df successfully saved | filename: dataset1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: dataset2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [5]:
df2.describe()
df1.describe()

,mode,ei,to,td,tf,vers,consumption
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,1.160000e+01,1.510916e+00,2.940000e+01,6.505000e+01,1.050000e+01,4.500000e+00,1.484044e+02
std,8.002501e+00,1.132716e+00,3.891838e+01,3.752776e+01,5.766282e+00,2.872282e+00,3.282216e+02
min,5.000000e+00,5.899000e-01,1.000000e+00,1.000000e-01,1.000000e+00,0.000000e+00,6.479167e-06
25%,5.000000e+00,9.330000e-01,3.000000e+00,3.257500e+01,5.750000e+00,2.000000e+00,6.713139e+00
50%,6.500000e+00,1.354955e+00,4.500000e+00,6.505000e+01,1.050000e+01,4.500000e+00,3.509333e+01
75%,2.100000e+01,1.354955e+00,5.500000e+01,9.752500e+01,1.525000e+01,7.000000e+00,1.428353e+02
max,2.600000e+01,4.814286e+00,1.200000e+02,1.300000e+02,2.000000e+01,9.000000e+00,6.292000e+03


In [46]:
from src.utils.file_utils import load_df
df1, df2 = load_df("dataset1.csv"), load_df("dataset2.csv")

## create neural nets

In [47]:
# def build_model():    
#     model = Sequential()
#     model.add(Dense(6, input_dim=5, activation="relu"))#hidden layer
#     model.add(Dense(10, activation="relu"))#hidden layer
#     model.add(Dense(1, activation='sigmoid'))#output layer

#     optimizer = RMSprop(0.001)
#     model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae', 'mape'])
#     return model

def build_model(num_features, num_classes):
    model = Sequential()
    model.add(Dense(32, input_shape=(num_features,), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer='adam', metrics=['accuracy'])
    return model

## train

In [54]:
def prepare_df(df, y_column):
    temp_df = df.copy()
    temp_df.sample(frac=1)
    
    y = to_categorical(
        temp_df[y_column].values)
    temp_df.drop(y_column, axis=1, inplace=True)
    features = list(temp_df.columns)
    
    x = temp_df.values
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)
    
    return x,y,features

x1, y1, features1 = prepare_df(df1, "vers")
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.20)
norm_train1, norm_test1 = pd.DataFrame(x_train1, columns=features1), pd.DataFrame(x_test1, columns=features1)
save_df(norm_train1, "norm_train1.csv")
save_df(norm_train1, "norm_test1.csv")

x2, y2, features2 = prepare_df(df2, "vers")
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.20)
norm_train2, norm_test2 = pd.DataFrame(x_train2, columns=features2), pd.DataFrame(x_test2, columns=features2)
save_df(norm_train2, "norm_train2.csv")
save_df(norm_train2, "norm_test2.csv")

df successfully saved | filename: norm_test2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [49]:
model1 = build_model(6,10)
history1 = model1.fit(x_train1, y_train1, epochs=10, validation_split=0.2, batch_size=128, verbose=2)

Train on 1664000 samples, validate on 416000 samples
Epoch 1/10
1664000/1664000 - 29s - loss: 1.1614 - accuracy: 0.5902 - val_loss: 0.7835 - val_accuracy: 0.7200
Epoch 2/10
1664000/1664000 - 27s - loss: 0.6724 - accuracy: 0.7616 - val_loss: 0.5938 - val_accuracy: 0.7868
Epoch 3/10
1664000/1664000 - 29s - loss: 0.5573 - accuracy: 0.7978 - val_loss: 0.5280 - val_accuracy: 0.8013
Epoch 4/10
1664000/1664000 - 27s - loss: 0.5017 - accuracy: 0.8145 - val_loss: 0.4876 - val_accuracy: 0.8228
Epoch 5/10
1664000/1664000 - 29s - loss: 0.4639 - accuracy: 0.8260 - val_loss: 0.4491 - val_accuracy: 0.8238
Epoch 6/10
1664000/1664000 - 28s - loss: 0.4364 - accuracy: 0.8339 - val_loss: 0.4355 - val_accuracy: 0.8340
Epoch 7/10
1664000/1664000 - 26s - loss: 0.4148 - accuracy: 0.8402 - val_loss: 0.4253 - val_accuracy: 0.8381
Epoch 8/10
1664000/1664000 - 23s - loss: 0.3979 - accuracy: 0.8459 - val_loss: 0.4107 - val_accuracy: 0.8379
Epoch 9/10
1664000/1664000 - 23s - loss: 0.3836 - accuracy: 0.8503 - val_lo

In [50]:
model2 = build_model(5,7)
history = model2.fit(x_train2, y_train2, epochs=20, validation_split=0.2, batch_size=128, verbose=2)

Train on 322560 samples, validate on 80640 samples
Epoch 1/15
322560/322560 - 4s - loss: 1.3205 - accuracy: 0.4360 - val_loss: 1.0828 - val_accuracy: 0.5200
Epoch 2/15
322560/322560 - 5s - loss: 0.9931 - accuracy: 0.5446 - val_loss: 0.9260 - val_accuracy: 0.6175
Epoch 3/15
322560/322560 - 4s - loss: 0.8885 - accuracy: 0.5886 - val_loss: 0.8630 - val_accuracy: 0.5528
Epoch 4/15
322560/322560 - 5s - loss: 0.8285 - accuracy: 0.6200 - val_loss: 0.7966 - val_accuracy: 0.6636
Epoch 5/15
322560/322560 - 5s - loss: 0.7851 - accuracy: 0.6387 - val_loss: 0.7638 - val_accuracy: 0.6720
Epoch 6/15
322560/322560 - 6s - loss: 0.7514 - accuracy: 0.6559 - val_loss: 0.7330 - val_accuracy: 0.6476
Epoch 7/15
322560/322560 - 5s - loss: 0.7249 - accuracy: 0.6625 - val_loss: 0.7109 - val_accuracy: 0.6681
Epoch 8/15
322560/322560 - 5s - loss: 0.7024 - accuracy: 0.6749 - val_loss: 0.6779 - val_accuracy: 0.6990
Epoch 9/15
322560/322560 - 4s - loss: 0.6786 - accuracy: 0.6972 - val_loss: 0.7025 - val_accuracy: 0.

## test

In [51]:
print(model1.evaluate(x_test1, y_test1))
print(model2.evaluate(x_test2, y_test2))
# [0.09895923781607832, 0.85723215]

520000/520000 [==============================] - 17s 33us/sample - loss: 0.3584 - accuracy: 0.8658
[0.3583500788943126, 0.8658192]
100800/100800 [==============================] - 3s 33us/sample - loss: 0.4695 - accuracy: 0.8580
[0.4694996028239765, 0.8580159]


In [35]:
preds = model2.predict_classes(x_test2)
# print(preds)
for i in range(len(y_test2[:10])):
    print(preds[i], list(y_test2[i]).index(1))

8 8
4 4
7 2
4 4
9 8
2 2
4 4
6 6
5 9
0 2


In [53]:
save_model(model1, "model1.h5")
save_model(model2, "model2.h5")

model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model1.h5
model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model2.h5
